# CV/ML Pipeline to extract highlights from videos using ML Models

With this notebook you can create an end-to-end CV/ML Pipeline using [GStreamer](gstreamer.freedesktop.org/) and run ML models to extract information from the frames. We'll use a Person detection + Pose estimation model based on Yolov7 to identify and track people in video files. With Gstreamer we can combine multiple feeds/cameras and create a mosaic of images. This helps us to accelerate the process.

First, deploy a pre-trained **Yolov7** to a SageMaker endpoint. Follow the instructions in [this notebook](01_Yolov7SageMakerInferentia.ipynb). Then, you can run this notebook.

## 1) Install dependencies

In [ ]:
# with this library we can build docker images and push them to ECR
%pip install sagemaker-studio-image-build

## 2) Initialize some variables

In [ ]:
import os
import io
import boto3
import tarfile
import sagemaker

sagemaker_session = sagemaker.Session()
bucket = sagemaker_session.default_bucket()
account_id = boto3.client('sts').get_caller_identity().get('Account')
region_name = sagemaker_session.boto_session.region_name

image_name='gstreamer'
image_tag="py3-1.0"
image_uri=f"{account_id}.dkr.ecr.{region_name}.amazonaws.com/{image_name}:{image_tag}"
print(f'Custom docker image: {image_uri}')

## 3) Build a custom docker container with additional libraries
**YOU DON"T NEED TO RUN** this section if you already did that before

In [ ]:
!pygmentize container_02/Dockerfile

### 3.1) Build and push the container image

In [ ]:
!sm-docker build container_02/ --repository $image_name:$image_tag

## 4) Create an application for processing our videos
This application will run inside a container executed by SageMaker Processing Jobs

### 4.1) Tracker object that makes use of ByteTrack
Source: https://github.com/ifzhang/ByteTrack  
This class assigns ids to detected objects and keeps track of them across multiple frames

In [ ]:
!pygmentize libs/tracker.py

### 4.2) CV Pipeline that wraps a GStreamer pipeline
Extend this class to create your own GStreamer pipeline solution

In [ ]:
!pygmentize libs/cvpipeline.py

### 4.3) SageMaker CV Pipeline
Extends a CVPipeline and invokes a SageMaker Endpoint for each frame

In [ ]:
!pygmentize libs/smcvpipeline.py

### 4.4) Main application
This script will parse all the parameters passed through SageMaker Processing jobs api and invoke the Gstreamer pipeline

In [ ]:
!pygmentize code_02/pipeline.py

### 4.5) Clone the correct version of ByteTrack
This library is required when object tracking is enabled

In [ ]:
import os
if not os.path.isdir('libs/bytetrack'):
    !git clone https://github.com/ifzhang/ByteTrack libs/bytetrack && \
        cd libs/bytetrack && git checkout d1bf019

## 5) Kick-off a SageMaker Processing job to process all our video files

In [ ]:
import sagemaker
from sagemaker.processing import ScriptProcessor
from sagemaker.processing import ProcessingInput, ProcessingOutput
from sagemaker.network import NetworkConfig

sagemaker_session = sagemaker.Session()
bucket = sagemaker_session.default_bucket()
print(f"s3://{bucket}/samples/")

### 5.1) Upload your .mp4 files to S3
If you don't have a video now and just want to run some tests, go to https://pixabay.com/videos/ or any other website which has video of people.

Download the **.mp4** as 720p (1280x720) files and upload them to the S3 path printed in the last cell (above).

Run the following command, then to make sure you uploaded the files:
```bash
aws s3 ls s3://<YOUR_BUCKET>/samples/  
```

### 5.2) Finally run the Processing Job

In [ ]:
import time
script_processor = ScriptProcessor(
    base_job_name=f'cv-pipeline-{int(time.time()*1000)}',
    image_uri=image_uri,
    role=sagemaker.get_execution_role(),
    instance_type='ml.c5.xlarge',
    instance_count=1,
    max_runtime_in_seconds=60 * 30,
    command=["/home/ec2-user/entrypoint.sh", "python3"],
    # for production it is important to define vpc_config and use a vpc_endpoint
    #vpc_config={
    #    'Subnets': ['<SUBNET1>', '<SUBNET2>'],
    #    'SecurityGroupIds': ['<SECURITYGROUP1>', '<DEFAULTSECURITYGROUP>']
    #}
)

script_processor.run(
    code='code_02/pipeline.py',
    inputs=[
        # always keep this input in the first place to avoid
        # issues with the pipe name
        ProcessingInput(
            source=f's3://{bucket}/samples',
            destination='/opt/ml/processing/input/data',            
            s3_input_mode='Pipe',
            s3_data_distribution_type='ShardedByS3Key'
        ),
        ProcessingInput(
            source='libs',
            destination='/opt/ml/processing/input/libs',
            s3_input_mode='File'
        )        
    ],
    outputs=[ProcessingOutput(
        source='/opt/ml/processing/output/predictions',
        destination=f's3://{bucket}/predictions/',
        s3_upload_mode='Continuous'
    )],
    arguments=[
        '--input-shape', '1280 720',
        '--endpoint-name', "yolov7-pose-inferentia",
        '--region-name', 'us-east-1'
    ]
)